# Neural Engineering Framework

In this notebook the NEF networks needed for Figure 4 are run. Developed for Python3.

In [ ]:
import nengo
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import seaborn
import sklearn.decomposition
import pandas as pd
%matplotlib inline

In [ ]:
N = 1000 #Number of neurons
D = 2    #Number of dimensions / latent variables

In [ ]:
filename = "../generatedData/oscillations.h5"

In [ ]:
#For each oscillator frequency
for f in range(1,26):
    print f
    
    #Unperturbed network
    model = nengo.Network(label="Oscillator")
    with model:
        A = nengo.Ensemble(N, dimensions=D, max_rates=nengo.dists.Uniform(80,120))
        tau = 0.01 #Synaptic time constant is 10 ms
        omega = f*np.pi*2
        mat = np.eye(2) + tau*np.array([[0, -omega],[omega, 0]])
        recurrent_connection = nengo.Connection(A, A, transform=mat, synapse=tau)
    sim = nengo.Simulator(model)
    eta = sim.data[A].encoders
    phi = sim.data[recurrent_connection].weights
    pd.DataFrame(eta, columns=("K1", "K2")).to_hdf(filename, "/nef/%d/original/K" % f)
    pd.DataFrame(phi.T, columns=("phi1", "phi2")).to_hdf(filename, "/nef/%d/original/phi" % f)

    #Inside-manifold perturbation (permute columns of encoder matrix)
    etaCols = eta[:,(1,0)]
    model = nengo.Network(label="Oscillator")
    with model:
        A = nengo.Ensemble(N, dimensions=D, encoders=etaCols, max_rates=nengo.dists.Uniform(80,120))
        tau = 0.01 #Synaptic time constant is 10 ms
        omega = f*np.pi*2
        mat = np.eye(2) + tau*np.array([[0, -omega],[omega, 0]])
        recurrent_connection = nengo.Connection(A, A, transform=mat, synapse=tau)
    sim = nengo.Simulator(model)
    phiCols = sim.data[recurrent_connection].weights
    pd.DataFrame(etaCols, columns=("K1", "K2")).to_hdf(filename, "/nef/%d/insideManifold/K" % f)
    pd.DataFrame(phiCols.T, columns=("phi1", "phi2")).to_hdf(filename, "/nef/%d/insideManifold/phi" % f)

    #Outside-manifold perturbation (permute rows of encoder matrix)
    permRowIndex = np.hstack((np.arange(500,1000), np.arange(500)))
    etaRows = eta[permRowIndex,:]
    model = nengo.Network(label="Oscillator")
    with model:
        A = nengo.Ensemble(N, dimensions=D, encoders=etaRows, max_rates=nengo.dists.Uniform(80,120))
        tau = 0.01 #Synaptic time constant is 10 ms
        omega = f*np.pi*2
        mat = np.eye(2) + tau*np.array([[0, -omega],[omega, 0]])
        recurrent_connection = nengo.Connection(A, A, transform=mat, synapse=tau)
    sim = nengo.Simulator(model)
    phiRows = sim.data[recurrent_connection].weights
    pd.DataFrame(etaRows, columns=("K1", "K2")).to_hdf(filename, "/nef/%d/outsideManifold/K" % f)
    pd.DataFrame(phiRows.T, columns=("phi1", "phi2")).to_hdf(filename, "/nef/%d/outsideManifold/phi" % f)

# Miscelleanous tests below

These parts are not needed to recreate the data

In [ ]:
scipy.stats.pearsonr(phiCol[(1,0),:].flat, phi.flat)

In [ ]:
eta

In [ ]:
model = nengo.Network(label="Oscillator")
with model:
    A = nengo.Ensemble(N, dimensions=K, max_rates=nengo.dists.Uniform(80,120))
    tau = 0.1 #Synaptic time constant is 10 ms
    input = nengo.Node(nengo.processes.Piecewise({0: [0,2], 0.05: [0, 0]}))
    input_connection = nengo.Connection(input, A, synapse=tau)
    omega = 3*np.pi*2
    mat = 1.1*np.eye(2) + tau*np.array([[0, -omega],[omega, 0]])
    recurrent_connection = nengo.Connection(A, A, transform=mat, synapse=tau)
    A_probe = nengo.Probe(A, synapse=tau)
    spikes_probe = nengo.Probe(A.neurons)
sim = nengo.Simulator(model)
sim.run(5)
eta = sim.data[A].encoders
phi = sim.data[recurrent_connection].weights

In [ ]:
plt.plot(sim.trange(), sim.data[A_probe][:,0])

In [ ]:
plt.plot(sim.trange(), sim.data[A_probe][:,1])

In [ ]:
sim.trange()[21]

In [ ]:
25*4*25 / 60.0 / 24.0